# Data Engineer @ Embat - Hiring Test

This notebook has several tasks to test the abilities of Data Engineer candidates @ Embat. Main topic is data analytics with Pyhton and SQL. We don't expect you to know how to solve all these tasks from memory, so we appreciate that you also note down all you had to google, so we get an idea of your thinking process. Also, its okay if you don't complete some tasks, but please note down your blockers.

## Part I - Data analytics with Python

(I.1) Load the sales.csv file into a pandas dataframe and answer the following questions:
* How many rows and colums do we have? How does our dataframe look like?
* How many unique values do we have in each column?
* Statistical values (min, max, avg, deviaton and percentiles) for the amount and the price of the coffee sold.
* Feel free to add any other information / metrics you may find interesting.

In [1]:
import pandas as pd
import numpy as np

# Load the sales.csv file into pandas dataframe
df = pd.read_csv("./sales.csv", parse_dates=["timestamp"])

# How many rows and columns?
print(f'{df.shape[0]} rows and {df.shape[1]} columns in the DataFrame')
df

1000 rows and 9 columns in the DataFrame


,name,surname,address,postal_code,city,timestamp,coffee_type,amount_kg,price_kg
0,Cleo,Melendez,"Calle U, nº 377",44175,Orrios,5/10/2022 20:23:47,8,"1,250","24,8"
1,Ryder,Mack,"Calle A, nº 320",37127,Espeja,31/01/2022 16:52:33,1,"0,500","34,8"
2,Yasir,Glover,"Calle T, nº 31",10192,Torremocha,31/10/2022 15:16:48,2,"0,250","20,4"
3,Eagan,Suarez,"Calle L, nº 164",50286,Villalba de Perejil,11/02/2022 3:16:39,1,"1,000","31,4"
4,Kimberly,Farmer,"Calle K, nº 229",48079,Errigoiti,22/02/2022 12:09:00,1,"0,750","24,2"
...,...,...,...,...,...,...,...,...,...
995,Demetrius,Adkins,"Calle T, nº 350",46081,Canals,24/10/2022 18:26:59,6,"5,000","27,9"
996,Stephen,Barlow,"Calle Q, nº 259",6042,Cheles,15/03/2022 4:30:30,5,"3,000","32,8"
997,Willa,Mueller,"Calle G, nº 244",14018,"Carpio, El",22/04/2022 0:52:59,5,"1,250","15,3"
998,Wyoming,Campos,"Calle G, nº 114",22162,Novales,27/12/2022 15:39:02,5,"3,000",31


In [2]:
# Taking a look to 'coffee_type' column. Not so clean
df['coffee_type'].unique()

array(['8', '1', '2', '3', '7', '4', '6', '5', nan, 'siete', 'seis'],
      dtype=object)

In [3]:
# Delete 3 rows with NaN values in 'coffee_type' column:
df.dropna(subset='coffee_type', inplace=True)

# And replacing some values:
df['coffee_type'] = df['coffee_type'].replace({'siete': '7', 'seis': '6'}).astype(int)

In [4]:
# checking again: 
df['coffee_type'].value_counts(dropna = False)

3    148
8    136
2    123
7    122
5    120
1    119
6    118
4    111
Name: coffee_type, dtype: int64

In [5]:
# How many unique values do we have in each column?
print ("UNIQUE VALUES")
print(df.nunique(dropna=False))

UNIQUE VALUES
name           666
surname        633
address        957
postal_code    995
city           997
timestamp      997
coffee_type      8
amount_kg       12
price_kg       201
dtype: int64


In [6]:
# Statistical values 
df[['amount_kg','price_kg']].describe()

# Percentiles are not calculated so it means that not all datapoints are numeric. In fact they are object data (strings and/or NaN).

,amount_kg,price_kg
count,997,994
unique,12,200
top,"1,250","24,8"
freq,122,10


In [7]:
# Taking a look to 'amount_kg' and 'price_kg' we can see 3 NaN in 'price_kg'
df[['amount_kg','price_kg']].isna().sum()

amount_kg    0
price_kg     3
dtype: int64

In [8]:
df.dropna(subset='price_kg', inplace=True)
df[['amount_kg','price_kg']].isna().sum()

amount_kg    0
price_kg     0
dtype: int64

In [9]:
# With both columns clean, we can change str to float in order to be able to calculate percentiles:
not_int_columns = ['amount_kg','price_kg']
for e in not_int_columns:
    df[e] = df[e].str.replace(",",".").astype(float)

In [10]:
df.amount_kg.describe()

# Strange max (5000.000) in the amount_kg. We may have some outliers.

count     994.000000
mean        9.870976
std       172.092504
min         0.250000
25%         0.750000
50%         1.250000
75%         3.000000
max      5000.000000
Name: amount_kg, dtype: float64

In [11]:
df.amount_kg.sort_values(ascending=False)

570    5000.00
581    1500.00
580    1500.00
870       5.00
96        5.00
        ...   
632       0.25
82        0.25
301       0.25
569       0.25
420       0.25
Name: amount_kg, Length: 994, dtype: float64

Three outliers detected in "amount_kg" column. We can fix it replacing values

In [12]:
df.loc[570,"amount_kg"] = 5.0
df.loc[581,"amount_kg"] = 1.5
df.loc[580,"amount_kg"] = 1.5

In [13]:
# Now we can check that the outliers are gone. Fixed.
df.amount_kg.sort_values(ascending=False)

805    5.00
620    5.00
848    5.00
853    5.00
657    5.00
       ... 
632    0.25
82     0.25
301    0.25
569    0.25
420    0.25
Name: amount_kg, Length: 994, dtype: float64

In [14]:
# So now we can have proper statistical values
df[['amount_kg','price_kg']].describe()

,amount_kg,price_kg
count,994.000000,994.000000
mean,1.830734,25.267907
std,1.443318,5.727261
min,0.250000,15.000000
25%,0.750000,20.400000
50%,1.250000,25.400000
75%,3.000000,30.300000
max,5.000000,35.000000


In [15]:
# Also we can check which column has NaN
count_nan_in_df = df.isna().sum()
print(count_nan_in_df)

name           0
surname        0
address        0
postal_code    0
city           0
timestamp      0
coffee_type    0
amount_kg      0
price_kg       0
dtype: int64


In [16]:
# Checking which rows has no numeric datapoints (NaN)
df_nans = df[['coffee_type','amount_kg','price_kg']]
count_nan_in_df = df_nans.isna().sum()
count_nan_in_df

coffee_type    0
amount_kg      0
price_kg       0
dtype: int64

In [17]:
# Checking each column type:
df.info()

# We should check 'timestamp' dtype in order to use the column as 'datetime'

<class 'pandas.core.frame.DataFrame'>
Int64Index: 994 entries, 0 to 999
Data columns (total 9 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   name         994 non-null    object 
 1   surname      994 non-null    object 
 2   address      994 non-null    object 
 3   postal_code  994 non-null    int64  
 4   city         994 non-null    object 
 5   timestamp    994 non-null    object 
 6   coffee_type  994 non-null    int64  
 7   amount_kg    994 non-null    float64
 8   price_kg     994 non-null    float64
dtypes: float64(2), int64(2), object(5)
memory usage: 109.9+ KB


### Checking and setting 'timestamp' column to datetime:

In [18]:
# Assign 'timestamp' to a variable so I can work with it independently
dates = df['timestamp']

In [19]:
# Trying to convert 'timestamp' column to datetime, I can raise some errors.
# Switching to 'coerce' I avoid the errors, convert them into NaT so I can later locate them in df
df2 = pd.to_datetime(dates, errors = 'coerce')

In [20]:
# Searching NaN in 'timestamp'
df2.isna().value_counts()

# 5 NaN found

False    989
True       5
Name: timestamp, dtype: int64

In [21]:
# Indexes which rows has NaN:
nan_index = df2[df2.isna()].index
df['timestamp'].loc[nan_index]

# Looking at the df data, we can see some wrong year values (year 3022) raising "OutOfBoundsDatetime". 

297    9/08/3022 22:47:07
461    10/08/3022 3:21:10
483    9/08/3022 22:00:10
626     9/08/3022 4:58:42
855     9/08/3022 0:41:09
Name: timestamp, dtype: object

In [22]:
# In case we want to fix that datapoints we could replace the wrong year doing it in advance as shown below:
for e in nan_index:
    valid_date = df['timestamp'][e].replace("3022","2022")
    df.loc[e,'timestamp'] = valid_date

In [23]:
# With all 'timestamp' datapoints in range, now we can convert the column to_datetime with no errors:
df['timestamp'] = pd.to_datetime(df['timestamp'], errors = 'raise')

(I.2) Now please load the types.csv and get the following data. Please show the country and coffee variety in the results, not just the type numbers:
* The total amount of Arabica coffee from Uganda sold in January 2022
* The average price of Arabica coffee from Brasil in Q1 2022.
* Same for Q2, Q3 and Q4 respectively.
* The total sales amount per source country.
* The total sales amount per shipping address province.
* In which province did we sell the higher cost orders on average?

In [24]:
# in order to merge the 2 dataframes, I should rename the shared column 'type'
df.columns = df.columns.str.replace('coffee_type', 'type')

In [25]:
# Load the types.csv file into pandas dataframe
df_types = pd.read_csv("./types.csv")
df_types

,type,country,variety,description
0,1,Kenia,Arabica,EXCEPTIONALLY DARK AND CREAMY COFFEE. INTENSIT...
1,2,Brasil,Arabica,POWERFUL AND CONTRASTING COFFEE. INTENSITY 10
2,3,Uganda,Arabica,HARMONIOUSLY BALANCED AND THICK BODY COFFEE. I...
3,4,Colombia,Arabica,POTENT AND ROASTED COFFEE. INTENSITY 10
4,5,Colombia,Robusta,FRUITY WITH FINE ACIDITY COFFEE. INTENSITY 5
5,6,Vietnam,Robusta,SWEET COFFEE AND CEREALS. INTENSITY 4
6,7,Brasil,Robusta,COFFEE WITH ROBUSTA MONSOON. INTENSITY 11
7,8,Indonesia,Robusta,WET-HULLED COFFEE. INTENSITY 8


In [26]:
left = df
right = df_types
result = pd.merge(left, right, on='type')
result

,name,surname,address,postal_code,city,timestamp,type,amount_kg,price_kg,country,variety,description
0,Cleo,Melendez,"Calle U, nº 377",44175,Orrios,2022-05-10 20:23:47,8,1.25,24.8,Indonesia,Robusta,WET-HULLED COFFEE. INTENSITY 8
1,Haley,Rivas,"Calle K, nº 369",19095,Condemios de Abajo,2022-02-25 12:22:12,8,0.75,29.2,Indonesia,Robusta,WET-HULLED COFFEE. INTENSITY 8
2,Andrew,Whitley,"Calle M, nº 196",47073,Herrín de Campos,2022-10-16 04:38:07,8,0.75,25.7,Indonesia,Robusta,WET-HULLED COFFEE. INTENSITY 8
3,Galvin,Houston,"Calle G, nº 355",19166,Málaga del Fresno,2022-08-31 17:58:53,8,5.00,20.3,Indonesia,Robusta,WET-HULLED COFFEE. INTENSITY 8
4,Hu,Brock,"Calle T, nº 300",39058,Rasines,2022-05-16 07:25:14,8,3.00,27.6,Indonesia,Robusta,WET-HULLED COFFEE. INTENSITY 8
...,...,...,...,...,...,...,...,...,...,...,...,...
989,Rana,Underwood,"Calle B, nº 154",4085,Somontín,2022-02-08 04:39:34,5,2.00,24.5,Colombia,Robusta,FRUITY WITH FINE ACIDITY COFFEE. INTENSITY 5
990,Baxter,Parker,"Calle T, nº 32",8219,Vilassar de Mar,2022-10-18 19:25:27,5,0.50,26.0,Colombia,Robusta,FRUITY WITH FINE ACIDITY COFFEE. INTENSITY 5
991,Stephen,Barlow,"Calle Q, nº 259",6042,Cheles,2022-03-15 04:30:30,5,3.00,32.8,Colombia,Robusta,FRUITY WITH FINE ACIDITY COFFEE. INTENSITY 5
992,Willa,Mueller,"Calle G, nº 244",14018,"Carpio, El",2022-04-22 00:52:59,5,1.25,15.3,Colombia,Robusta,FRUITY WITH FINE ACIDITY COFFEE. INTENSITY 5


In [27]:
# The total amount of Arabica coffee from Uganda sold in January 2022:

answer_1 = result[(result['country']=='Uganda') & (result['variety']=='Arabica') & (result['timestamp'].dt.month==1)]
col = ['timestamp','type', 'variety', 'country', 'amount_kg']
out = result[col].groupby(by = ['country', 'type', 'variety', answer_1['timestamp'].dt.month]).agg({'amount_kg':'sum'})
out

,,,,amount_kg
country,type,variety,timestamp,
Uganda,3,Arabica,1.0,16.25


In [28]:
# The average price of Arabica coffee from Brasil in Q1-Q2-Q3-Q4 2022:

def quarter(quarter):
    answer_2 = result[(result['country']=='Brasil') & (result['timestamp'].dt.quarter==e) & (result['variety']=='Arabica')]
    col = ['timestamp','type', 'variety', 'country', 'price_kg']
    out = result[col].groupby(by = ['country', 'type', 'variety', answer_2['timestamp'].dt.quarter]).agg({'price_kg':'mean'})
    return(display(out))

for e in range(1,5):
    print(f'Average price Quarter {e}')
    quarter(e)

Average price Quarter 1


,,,,price_kg
country,type,variety,timestamp,
Brasil,2,Arabica,1.0,26.268966


Average price Quarter 2


,,,,price_kg
country,type,variety,timestamp,
Brasil,2,Arabica,2.0,26.769231


Average price Quarter 3


,,,,price_kg
country,type,variety,timestamp,
Brasil,2,Arabica,3.0,23.830769


Average price Quarter 4


,,,,price_kg
country,type,variety,timestamp,
Brasil,2,Arabica,4.0,24.365517


In [29]:
# The total sales amount per source country:

# Let's create a new column called 'sales_amount':
result['sales_amount'] = result['amount_kg'] * result['price_kg']

In [30]:
col = ['country', 'variety', 'type', 'sales_amount']
out = result[col].groupby(by = ['country', 'variety', 'type']).agg({'sales_amount':'sum'})
out

sales_amount
country   variety type              
Brasil    Arabica 2         5692.875
          Robusta 7         4770.975
Colombia  Arabica 4         5528.950
          Robusta 5         6039.550
Indonesia Robusta 8         6352.675
Kenia     Arabica 1         4805.200
Uganda    Arabica 3         7440.325
Vietnam   Robusta 6         5482.475

### Provinces:
How to create a new column with province names:
- A dictionary {k:v} k=postal_code, v=provincia
- A function formating postal codes (5 digits, filling zeros)
- A function assigns province names according to the postal codes in the dict.

In [31]:
# I create a dict with postal codes and provinces out of a postal_codes.csv
import csv
codes_dict = {}
with open('postal_codes.csv', newline='') as pcodes_cfile:
    reader = csv.DictReader(pcodes_cfile)
    for row in reader:
        codes_dict[row['codigo']] = row['provincia'][1:]
print (codes_dict)


{'02': 'Albacete', '03': 'Alicante', '04': 'Almería', '01': 'Álava', '33': 'Asturias', '05': 'Ávila', '06': 'Badajoz', '07': 'Baleares', '08': 'Barcelona', '48': 'Bizkaia', '09': 'Burgos', '10': 'Cáceres', '11': 'Cádiz', '39': 'Cantabria', '12': 'Castellón', '13': 'Ciudad Real', '14': 'Córdoba', '15': 'Coruña', '16': 'Cuenca', '20': 'Gipuzkoa', '17': 'Girona', '18': 'Granada', '19': 'Guadalajara', '21': 'Huelva', '22': 'Huesca', '23': 'Jaén', '24': 'León', '25': 'Lleida', '27': 'Lugo', '28': 'Madrid', '29': 'Málaga', '30': 'Murcia', '31': 'Navarra', '32': 'Ourense', '34': 'Palencia', '35': 'Las Palmas', '36': 'Pontevedra', '26': 'La Rioja', '37': 'Salamanca', '38': 'Santa Cruz de Tenerife', '40': 'Segovia', '41': 'Sevilla', '42': 'Soria', '43': 'Tarragona', '44': 'Teruel', '45': 'Toledo', '46': 'Valencia', '47': 'Valladolid', '49': 'Zamora', '50': 'Zaragoza', '51': 'Ceuta', '52': 'Melilla'}


In [32]:
# Change 'postal_code' format to str:
result['postal_code'] = result['postal_code'].astype(str)

In [33]:
# Fill postal_code string with zeros till 5 digits
def codes_zeros (e):
    if len(str(e))>5:
        e = "0"
    e = str(e).zfill(5)
    return(e)

# Return provinces from codes_dict based on postal codes:
def province (e):
    return codes_dict.get(e[:2], "Not_Found" )

In [34]:
# Modify 'postal_code' 
result['postal_code'] = result['postal_code'].apply(codes_zeros)

# Create a new column 'province' in the 'result' dataframe
result['province'] = result['postal_code'].apply(province)

In [35]:
# The total sales amount per shipping address province.

# We need to create a new column for that and infer the province from the postal code
col = ['province', 'variety', 'type', 'sales_amount']
out = result[col].groupby(by = ['province']).agg({'sales_amount':'sum'}).sort_values(by="sales_amount",ascending=False)
#out.sales_amount.sort_values(ascending=False)
out.head()


,sales_amount
province,
Burgos,2117.850
Barcelona,2087.700
Cuenca,1841.750
Lleida,1731.600
Zamora,1705.125


In [36]:
# Some postal_codes were not introduced properly in the df
result[(result == 'Not_Found').any(axis=1)]

,name,surname,address,postal_code,city,timestamp,type,amount_kg,price_kg,country,variety,description,sales_amount,province
268,Chancellor,Burris,"Calle O, nº 8",00000,Colmenarejo,2022-05-28 21:56:52,2,0.25,25.0,Brasil,Arabica,POWERFUL AND CONTRASTING COFFEE. INTENSITY 10,6.250,Not_Found
319,Olivia,David,"Calle J, nº 205",00000,Medinaceli,2022-07-22 04:19:58,2,2.00,22.8,Brasil,Arabica,POWERFUL AND CONTRASTING COFFEE. INTENSITY 10,45.600,Not_Found
551,Reed,Meyer,"Calle T, nº 10",00000,Terrer,2022-05-23 21:02:37,7,0.75,28.5,Brasil,Robusta,COFFEE WITH ROBUSTA MONSOON. INTENSITY 11,21.375,Not_Found
689,Griffin,Wiley,"Calle C, nº 193",00000,Beade,2022-08-16 18:22:57,4,2.00,34.8,Colombia,Arabica,POTENT AND ROASTED COFFEE. INTENSITY 10,69.600,Not_Found
859,Shelley,Sykes,"Calle K, nº 231",99999,Loiu,2022-11-04 15:54:30,6,5.00,33.8,Vietnam,Robusta,SWEET COFFEE AND CEREALS. INTENSITY 4,169.000,Not_Found
973,Bruce,Kirby,"Calle N, nº 147",99999,Bimenes,2022-01-20 15:57:57,5,0.50,18.1,Colombia,Robusta,FRUITY WITH FINE ACIDITY COFFEE. INTENSITY 5,9.050,Not_Found


In [37]:
# In which province did we sell the higher cost orders on average?
col = ['province', 'variety', 'type', 'sales_amount']
out = result[col].groupby(by = ['province']).agg({'sales_amount':'mean'})
out.sort_values(['sales_amount'], ascending = False).head()

,sales_amount
province,
Santa Cruz de Tenerife,65.400000
Huelva,63.937500
Cuenca,63.508621
Alicante,62.295000
Lleida,61.842857


(I.3) Using the dictionary below -where the keys are the types and the values the commercial names-, please add a new column to the types dataframe showing this commercial name.

In [38]:
coffee = {'1': 'Kenia (A)',
          '2': 'Brasil (A)',
          '3': 'Uganda (A)',
          '4': 'Colombia (A)',
          '5': 'Colombia (R)',
          '6': 'Vietnam (R)',
          '7': 'Brasil (R)',
          '8': 'Indonesia (R)'}

In [39]:
# As dictionary keys are "str" but df['type'] are "int", so I modify the dict changing the keys type to int:
coffee = {int(k):v for k,v in coffee.items()}

# Insert blocks here to complete the task.
df_types['commercial_name'] = df_types['type'].map(coffee)
df_types

,type,country,variety,description,commercial_name
0,1,Kenia,Arabica,EXCEPTIONALLY DARK AND CREAMY COFFEE. INTENSIT...,Kenia (A)
1,2,Brasil,Arabica,POWERFUL AND CONTRASTING COFFEE. INTENSITY 10,Brasil (A)
2,3,Uganda,Arabica,HARMONIOUSLY BALANCED AND THICK BODY COFFEE. I...,Uganda (A)
3,4,Colombia,Arabica,POTENT AND ROASTED COFFEE. INTENSITY 10,Colombia (A)
4,5,Colombia,Robusta,FRUITY WITH FINE ACIDITY COFFEE. INTENSITY 5,Colombia (R)
5,6,Vietnam,Robusta,SWEET COFFEE AND CEREALS. INTENSITY 4,Vietnam (R)
6,7,Brasil,Robusta,COFFEE WITH ROBUSTA MONSOON. INTENSITY 11,Brasil (R)
7,8,Indonesia,Robusta,WET-HULLED COFFEE. INTENSITY 8,Indonesia (R)


(I.4) Finally, create a new sales dataframe with all the data cleansing processes you may need to upload this data to a sql database.

In [40]:
sales = result[['name','surname','address','postal_code','city','province','timestamp','type','amount_kg','price_kg','sales_amount']]
sales

,name,surname,address,postal_code,city,province,timestamp,type,amount_kg,price_kg,sales_amount
0,Cleo,Melendez,"Calle U, nº 377",44175,Orrios,Teruel,2022-05-10 20:23:47,8,1.25,24.8,31.000
1,Haley,Rivas,"Calle K, nº 369",19095,Condemios de Abajo,Guadalajara,2022-02-25 12:22:12,8,0.75,29.2,21.900
2,Andrew,Whitley,"Calle M, nº 196",47073,Herrín de Campos,Valladolid,2022-10-16 04:38:07,8,0.75,25.7,19.275
3,Galvin,Houston,"Calle G, nº 355",19166,Málaga del Fresno,Guadalajara,2022-08-31 17:58:53,8,5.00,20.3,101.500
4,Hu,Brock,"Calle T, nº 300",39058,Rasines,Cantabria,2022-05-16 07:25:14,8,3.00,27.6,82.800
...,...,...,...,...,...,...,...,...,...,...,...
989,Rana,Underwood,"Calle B, nº 154",04085,Somontín,Almería,2022-02-08 04:39:34,5,2.00,24.5,49.000
990,Baxter,Parker,"Calle T, nº 32",08219,Vilassar de Mar,Barcelona,2022-10-18 19:25:27,5,0.50,26.0,13.000
991,Stephen,Barlow,"Calle Q, nº 259",06042,Cheles,Badajoz,2022-03-15 04:30:30,5,3.00,32.8,98.400
992,Willa,Mueller,"Calle G, nº 244",14018,"Carpio, El",Córdoba,2022-04-22 00:52:59,5,1.25,15.3,19.125


In [41]:
# Saving a copy of the clean 'sales' dataframe in .csv
# so we can load it later as a table in SQLite
sales.to_csv('sales_up.csv')

# We can also create a SQLite table out of our clean upgraded dataframe, as shown 3 cells below.

## Part II - SQL Database

(II.1) You need to create a SQLite database with two tables and load the data from the attached csv files. Make sure all the data has its appropiate data type and that you upload it correctly. First two code blocks import the sqlite module and create the database and its connection.

In [42]:
import csv
import sqlite3

In [43]:
# Connecting to the embat database
conn = sqlite3.connect('embat_data')

# Creating a cursor object to execute SQL queries on a database table
c = conn.cursor()

In [44]:
# Let's create a SQLite table out of our clean (fixed timestamps, dropped NaNs...) and 
# upgraded dataframe (including 'provinces', 'timestamp', 'sales_amount'...)
sales.to_sql(name='sales_update', con=conn)
conn.close()

In [45]:
# Table Definition
create_table =  '''CREATE TABLE IF NOT EXISTS sales(
    name TEXT NOT NULL,
    surname TEXT NOT NULL,
    address TEXT NOT NULL,
    postal_code TEXT NOT NULL,
    city TEXT NOT NULL,
    timestamp TEXT NOT NULL,
    type INTEGER NOT NULL,
    amount_kg REAL NOT NULL,
    price_kg REAL NOT NULL
    );
    '''

create_table2 =  '''CREATE TABLE IF NOT EXISTS coffee_types(
    type INTEGER NOT NULL,
    country TEXT NOT NULL,
    variety TEXT NOT NULL,
    description TEXT NOT NULL
    );
    '''

In [46]:
conn = sqlite3.connect('embat_data')
c = conn.cursor()

# Creating the tables into the database
c.execute(create_table)
c.execute(create_table2)

# Opening the 'sales.csv' and 'types.csv' file
file = open('sales.csv')
file2 = open('types.csv')

# Reading the contents of the 'sales.cs'v and 'types.csv' file
contents = csv.reader(file)
contents2 = csv.reader(file2)
 
# SQL query to insert data into the 'sales' table
insert_sales = "INSERT INTO sales (name, surname, address, postal_code, city, timestamp, type, amount_kg, price_kg) VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?)"

# SQL query to insert data into the 'coffee_types' table
insert_coffe_types = "INSERT INTO coffee_types (type, country, variety, description) VALUES(?, ?, ?, ?)"
 
# Importing the contents of the files into our 'sales' and 'coffe_types' table
c.executemany(insert_sales, contents)
c.executemany(insert_coffe_types, contents2)
 
# SQL query to retrieve all data from the sales table to verify that the
# data of the csv file has been successfully inserted into the table
select_5 = "SELECT * FROM sales"
rows = c.execute(select_5).fetchmany(5)

select_5 = "SELECT * FROM coffee_types"
rows2 = c.execute(select_5).fetchmany(5)
 
# Output to the console screen
for r in rows:
    print(r)
print("\n")
for r2 in rows2:
    print(r2)

# Committing changes and closing the database connection:
conn.commit()
conn.close()

('name', 'surname', 'address', 'postal_code', 'city', 'timestamp', 'coffee_type', 'amount_kg', 'price_kg')
('Cleo', 'Melendez', 'Calle U, nº 377', '44175', 'Orrios', '5/10/2022 20:23:47', 8, '1,250', '24,8')
('Ryder', 'Mack', 'Calle A, nº 320', '37127', 'Espeja', '31/01/2022 16:52:33', 1, '0,500', '34,8')
('Yasir', 'Glover', 'Calle T, nº 31', '10192', 'Torremocha', '31/10/2022 15:16:48', 2, '0,250', '20,4')
('Eagan', 'Suarez', 'Calle L, nº 164', '50286', 'Villalba de Perejil', '11/02/2022 3:16:39', 1, '1,000', '31,4')


('type', 'country', 'variety', 'description')
(1, 'Kenia', 'Arabica', 'EXCEPTIONALLY DARK AND CREAMY COFFEE. INTENSITY  13')
(2, 'Brasil', 'Arabica', 'POWERFUL AND CONTRASTING COFFEE. INTENSITY  10')
(3, 'Uganda', 'Arabica', 'HARMONIOUSLY BALANCED AND THICK BODY COFFEE. INTENSITY  8')
(4, 'Colombia', 'Arabica', 'POTENT AND ROASTED COFFEE. INTENSITY  10')


(II.2) Answer the same questions solved in (I.1) and (I.2) with sql queries

(I.1) Answer the following questions related with sales.csv:
* How many rows and colums do we have? How does our dataframe look like?
* How many unique values do we have in each column?
* Statistical values (min, max, avg, deviaton and percentiles) for the amount and the price of the coffee sold.
* Feel free to add any other information / metrics you may find interesting.

In [47]:
conn = sqlite3.connect('embat_data')
c = conn.cursor()

# how many rows do we have?
c.execute('SELECT count(*) FROM sales')
rows = c.fetchone()

# how many columns do we have?
data = c.execute('SELECT * FROM sales')
col = len(data.description)

print(f'{rows[0]} rows, {col} columns')

conn.commit() 
conn.close()

1001 rows, 9 columns


In [48]:
conn = sqlite3.connect('embat_data')
c = conn.cursor()

# I can keep columns names in a list for:
col_names = []
for column in data.description:
    col_names.append(column[0])

# How many unique values do we have in each column?
print ('UNIQUE VALUES PER COLUMN')
for col in col_names:
    c.execute(f'SELECT COUNT(DISTINCT({col})) FROM sales')
    uniq_values = c.fetchone()[0]
    cadena = '{:<15}{:>5}'.format(col, uniq_values)
    print(cadena)

conn.commit() 
conn.close()
    

UNIQUE VALUES PER COLUMN
name             669
surname          635
address          961
postal_code      999
city            1001
timestamp       1001
type              12
amount_kg         13
price_kg         202


In [49]:
conn = sqlite3.connect('embat_data')
c = conn.cursor()

# Statistical values (min, max, avg, deviaton and percentiles) for the amount and the price of the coffee sold.
columnas = ["amount_kg", "price_kg"]

# Preparing the queries:

def statistics(col):
    print(f'\n{col.upper()}')
    q_min = (f'SELECT MIN({col}) FROM sales', 'min')
    q_max = (f'SELECT MAX({col}) FROM sales', 'max')
    q_mean = (f'SELECT ROUND(AVG({col}),3) FROM sales', 'avg')
    calcs = [q_min, q_max, q_mean]
    for calc in calcs:
        c.execute(calc[0])
        value = c.fetchone()[0]
        out = '{:<15}{:>15}'.format(calc[1], value)
        print(out)

for col in columnas:
    statistics(col)


conn.commit() 
conn.close()



AMOUNT_KG
min                     1500.0
max                  amount_kg
avg                      9.573

PRICE_KG
min                       15.0
max                   price_kg
avg                       24.7


(I.2) Now please load the types.csv and get the following data. Please show the country and coffee variety in the results, not just the type numbers:

In [50]:
# The total amount of Arabica coffee from Uganda sold in January 2022
# WIP - It's working in 'sales' table (with a trick managing dates), instead of 'sales_update'
conn = sqlite3.connect('embat_data')
c = conn.cursor()

query = '''
        SELECT 
            sales.type, 
            country, 
            variety, 
            SUM (amount_kg)
        FROM 
            sales
        JOIN 
            coffee_types
        ON 
            sales.type = coffee_types.type
        WHERE
            coffee_types.variety = 'Arabica' and
            coffee_types.country = 'Uganda' and
            timestamp LIKE '%/01/%'
            ;
        '''

out = c.execute(query)
for e in out:
    print(e)

conn.commit() 
conn.close()


(3, 'Uganda', 'Arabica', 27.0)


In [51]:
# The average price of Arabica coffee from Brasil in Q1, Q2, Q3, Q4 2022.
# WIP - Not yet fully solved. Managing timestamps missing.
conn = sqlite3.connect('embat_data')
c = conn.cursor()

query = '''
        SELECT 
            sales_update.type, 
            country, 
            variety, 
            AVG (price_kg)
        FROM 
            sales_update
        JOIN 
            coffee_types
        ON 
            sales_update.type = coffee_types.type
        WHERE
            coffee_types.variety = 'Arabica' and
            coffee_types.country = 'Brasil'
            ;
        '''
#timestamp LIKE '%/01/%'

out = c.execute(query)
for e in out:
    print(e)

conn.commit() 
conn.close()


(2, 'Brasil', 'Arabica', 25.15284552845529)


In [52]:
conn = sqlite3.connect('embat_data')
c = conn.cursor()

query = '''
        SELECT
            *
        FROM
            sales_update
        ;
        '''
pd.read_sql_query(query, conn)

,index,name,surname,address,postal_code,city,province,timestamp,type,amount_kg,price_kg,sales_amount
0,0,Cleo,Melendez,"Calle U, nº 377",44175,Orrios,Teruel,2022-05-10 20:23:47,8,1.25,24.8,31.000
1,1,Haley,Rivas,"Calle K, nº 369",19095,Condemios de Abajo,Guadalajara,2022-02-25 12:22:12,8,0.75,29.2,21.900
2,2,Andrew,Whitley,"Calle M, nº 196",47073,Herrín de Campos,Valladolid,2022-10-16 04:38:07,8,0.75,25.7,19.275
3,3,Galvin,Houston,"Calle G, nº 355",19166,Málaga del Fresno,Guadalajara,2022-08-31 17:58:53,8,5.00,20.3,101.500
4,4,Hu,Brock,"Calle T, nº 300",39058,Rasines,Cantabria,2022-05-16 07:25:14,8,3.00,27.6,82.800
...,...,...,...,...,...,...,...,...,...,...,...,...
989,989,Rana,Underwood,"Calle B, nº 154",04085,Somontín,Almería,2022-02-08 04:39:34,5,2.00,24.5,49.000
990,990,Baxter,Parker,"Calle T, nº 32",08219,Vilassar de Mar,Barcelona,2022-10-18 19:25:27,5,0.50,26.0,13.000
991,991,Stephen,Barlow,"Calle Q, nº 259",06042,Cheles,Badajoz,2022-03-15 04:30:30,5,3.00,32.8,98.400
992,992,Willa,Mueller,"Calle G, nº 244",14018,"Carpio, El",Córdoba,2022-04-22 00:52:59,5,1.25,15.3,19.125


In [53]:
# The total sales amount per source country.

conn = sqlite3.connect('embat_data')
c = conn.cursor()

query = '''
        SELECT
            country,
            SUM(sales_amount) AS total_sales_amount
        FROM
            sales_update
        JOIN
            coffee_types ON sales_update.type = coffee_types.type
        GROUP BY
            country
        ;
        '''
pd.read_sql_query(query, conn)

,country,total_sales_amount
0,Brasil,10463.850
1,Colombia,11568.500
2,Indonesia,6352.675
3,Kenia,4805.200
4,Uganda,7440.325
5,Vietnam,5482.475


In [54]:
# The total sales amount per shipping address province.
# Using 'sales_update' table.

conn = sqlite3.connect('embat_data')
c = conn.cursor()
query = '''
        SELECT  
            province,
            SUM(sales_amount)
        FROM 
            sales_update
        GROUP BY
            province
        ;
        '''
out = c.execute(query)
for row in out:
    print (row)
conn.commit()
conn.close() 


('Albacete', 302.19999999999993)
('Alicante', 934.4250000000001)
('Almería', 672.9250000000001)
('Asturias', 515.225)
('Badajoz', 864.7000000000002)
('Baleares', 452.40000000000003)
('Barcelona', 2087.7000000000003)
('Bizkaia', 629.4749999999999)
('Burgos', 2117.85)
('Cantabria', 514.5500000000001)
('Castellón', 712.85)
('Ciudad Real', 562.575)
('Coruña', 530.575)
('Cuenca', 1841.7500000000002)
('Cáceres', 1079.975)
('Cádiz', 268.475)
('Córdoba', 294.02500000000003)
('Gipuzkoa', 598.2)
('Girona', 1193.4999999999998)
('Granada', 992.725)
('Guadalajara', 1406.25)
('Huelva', 639.375)
('Huesca', 898.825)
('Jaén', 690.9)
('La Rioja', 1276.925)
('Las Palmas', 169.95)
('León', 1142.1499999999999)
('Lleida', 1731.6)
('Lugo', 431.5)
('Madrid', 1163.225)
('Melilla', 34.375)
('Murcia', 298.19999999999993)
('Málaga', 487.275)
('Navarra', 1389.775)
('Not_Found', 320.875)
('Ourense', 619.2249999999999)
('Palencia', 1203.1750000000002)
('Pontevedra', 175.125)
('Salamanca', 1626.7749999999999)
('Santa

In [55]:
# In which province did we sell the higher cost orders on average?
# Using 'sales_update' table.

conn = sqlite3.connect('embat_data')
c = conn.cursor()
query = '''
        SELECT  
            province,
            AVG(sales_amount) AS mean
        FROM 
            sales_update
        GROUP BY
            province
        ORDER BY mean DESC
        ;
        '''
out = c.execute(query)
for id, row in enumerate(out):
    if id<5:
        print(row)

conn.commit()
conn.close() 

('Santa Cruz de Tenerife', 65.4)
('Huelva', 63.9375)
('Cuenca', 63.50862068965518)
('Alicante', 62.295)
('Lleida', 61.84285714285714)


In [56]:
# TEST WITH sales_update SQLite table (works)
conn = sqlite3.connect('embat_data')
c = conn.cursor()

query = '''
        SELECT * FROM sales_update
        ;
        '''

out = c.execute(query)
for r in out:
    print(r)
conn.commit()
conn.close() 

(0, 'Cleo', 'Melendez', 'Calle U, nº 377', '44175', 'Orrios', 'Teruel', '2022-05-10 20:23:47', 8, 1.25, 24.8, 31.0)
(1, 'Haley', 'Rivas', 'Calle K, nº 369', '19095', 'Condemios de Abajo', 'Guadalajara', '2022-02-25 12:22:12', 8, 0.75, 29.2, 21.9)
(2, 'Andrew', 'Whitley', 'Calle M, nº 196', '47073', 'Herrín de Campos', 'Valladolid', '2022-10-16 04:38:07', 8, 0.75, 25.7, 19.275)
(3, 'Galvin', 'Houston', 'Calle G, nº 355', '19166', 'Málaga del Fresno', 'Guadalajara', '2022-08-31 17:58:53', 8, 5.0, 20.3, 101.5)
(4, 'Hu', 'Brock', 'Calle T, nº 300', '39058', 'Rasines', 'Cantabria', '2022-05-16 07:25:14', 8, 3.0, 27.6, 82.80000000000001)
(5, 'Damian', 'Yang', 'Calle V, nº 22', '48913', 'Zierbena', 'Bizkaia', '2022-06-13 02:00:27', 8, 4.0, 28.7, 114.8)
(6, 'Jason', 'Harding', 'Calle Z, nº 89', '32069', 'Ribadavia', 'Ourense', '2022-03-28 07:14:15', 8, 4.0, 17.0, 68.0)
(7, 'Amir', 'Barnes', 'Calle D, nº 52', '49024', 'Bóveda de Toro, La', 'Zamora', '2022-03-23 03:52:27', 8, 2.0, 28.1, 56.2)
(8

(II.3) Get the following data, each of them just with one sql query if possible:
* For the first 10 sales of the year, get their price per kilo and how does that compare to the average price per kilo in that province.


In [57]:
# From now on, I'm going to use the updated csv (sales_up.csv) that was created in the first part of this test.
# This time, I'll make the queries from pandas with the <pd.read_sql_query()> method.

# Load the sales_up.csv file into pandas dataframe
df = pd.read_csv("./sales_up.csv", parse_dates=["timestamp"])

In [58]:
conn = sqlite3.connect('embat_data')

query = '''
        SELECT
            *
        FROM
            sales_update
        ;
        '''

pd.read_sql_query(query, conn)

,index,name,surname,address,postal_code,city,province,timestamp,type,amount_kg,price_kg,sales_amount
0,0,Cleo,Melendez,"Calle U, nº 377",44175,Orrios,Teruel,2022-05-10 20:23:47,8,1.25,24.8,31.000
1,1,Haley,Rivas,"Calle K, nº 369",19095,Condemios de Abajo,Guadalajara,2022-02-25 12:22:12,8,0.75,29.2,21.900
2,2,Andrew,Whitley,"Calle M, nº 196",47073,Herrín de Campos,Valladolid,2022-10-16 04:38:07,8,0.75,25.7,19.275
3,3,Galvin,Houston,"Calle G, nº 355",19166,Málaga del Fresno,Guadalajara,2022-08-31 17:58:53,8,5.00,20.3,101.500
4,4,Hu,Brock,"Calle T, nº 300",39058,Rasines,Cantabria,2022-05-16 07:25:14,8,3.00,27.6,82.800
...,...,...,...,...,...,...,...,...,...,...,...,...
989,989,Rana,Underwood,"Calle B, nº 154",04085,Somontín,Almería,2022-02-08 04:39:34,5,2.00,24.5,49.000
990,990,Baxter,Parker,"Calle T, nº 32",08219,Vilassar de Mar,Barcelona,2022-10-18 19:25:27,5,0.50,26.0,13.000
991,991,Stephen,Barlow,"Calle Q, nº 259",06042,Cheles,Badajoz,2022-03-15 04:30:30,5,3.00,32.8,98.400
992,992,Willa,Mueller,"Calle G, nº 244",14018,"Carpio, El",Córdoba,2022-04-22 00:52:59,5,1.25,15.3,19.125


In [59]:
query = '''
        SELECT
            province,
            price_kg,
            round (AVG (price_kg) OVER(PARTITION BY province) , 3) AS avgpp,
            round (price_kg / AVG (price_kg) OVER(PARTITION BY province) , 3) AS "%"
        FROM
            sales_update
        ORDER BY
            timestamp
        ;
        '''

pd.read_sql_query(query, conn).head(10)

,province,price_kg,avgpp,%
0,Huelva,17.8,26.350,0.676
1,Zaragoza,21.5,24.518,0.877
2,Valladolid,32.3,24.193,1.335
3,Zamora,25.3,25.031,1.011
4,Badajoz,31.5,26.438,1.191
5,Coruña,22.8,23.008,0.991
6,Barcelona,24.8,25.074,0.989
7,Zamora,18.6,25.031,0.743
8,Zaragoza,33.5,24.518,1.366
9,Valladolid,15.2,24.193,0.628



* Show all sales shipped to Madrid province with their date and amount (kg) of coffee sold, and for each of them show also the total amount of coffee sold in Madrid that same month.


In [60]:
query = '''
        SELECT
            province,
            STRFTIME("%Y-%m-%d", timestamp) AS date,
            amount_kg,
            SUM(amount_kg) OVER(PARTITION BY STRFTIME("%m", timestamp)) AS total_month
        FROM
            sales_update
        WHERE province = "Madrid"
        ;
        '''
pd.read_sql_query(query, conn)

,province,date,amount_kg,total_month
0,Madrid,2022-01-22,0.75,0.75
1,Madrid,2022-02-04,5.00,6.25
2,Madrid,2022-02-22,1.25,6.25
3,Madrid,2022-03-26,5.00,5.00
4,Madrid,2022-04-15,0.75,4.75
5,Madrid,2022-04-24,4.00,4.75
6,Madrid,2022-05-04,3.00,4.25
7,Madrid,2022-05-20,1.25,4.25
8,Madrid,2022-06-18,3.00,3.00
9,Madrid,2022-07-08,1.00,3.50



* Similar to the previous, show all sales shipped to Madrid province with their date and amount (kg) of coffee sold. Order the sales chronologically by month (all sales in january, then all sales in february, etc.) but also from higher to lower amount sold within each month.


In [61]:
query = '''
        SELECT
            province,
            STRFTIME("%m", timestamp) AS month,
            amount_kg,
            SUM(amount_kg) OVER(PARTITION BY STRFTIME("%m", timestamp)) AS total_month
        FROM
            sales_update
        WHERE province = "Madrid"
        ORDER BY month, amount_kg DESC
        ;
        '''
pd.read_sql_query(query, conn)

,province,month,amount_kg,total_month
0,Madrid,01,0.75,0.75
1,Madrid,02,5.00,6.25
2,Madrid,02,1.25,6.25
3,Madrid,03,5.00,5.00
4,Madrid,04,4.00,4.75
5,Madrid,04,0.75,4.75
6,Madrid,05,3.00,4.25
7,Madrid,05,1.25,4.25
8,Madrid,06,3.00,3.00
9,Madrid,07,1.50,3.50
